# Lab 9 - Dense Neural Network
## Sudhan Shrestha [8889436]

1. Using Keras Datasets, obtain the fMNIST data set. Explore and describe it as you would do with any new dataset that you start working with (2 points). 

2. Train a dense neural network using Keras (2 points)

3. Evaluate its accuracy. Report accuracy, confusion metric, explore specific examples of incorrect classification, and drew some conclusions from this exploration (2 points)